Pierre Navaro - [Institut de Recherche Mathématique de Rennes](https://irmar.univ-rennes1.fr) - [CNRS](http://www.cnrs.fr/)

https://github.com/pnavaro/big-data/blob/master/02.Containers.ipynb

[Display on nbviewer](http://nbviewer.jupyter.org/github/pnavaro/big-data/blob/master/02.Containers.ipynb)

- All approaches in notebook 01 load all the data into memory. A very large file might fill up memory. 
- Counting words in each line is totally independent of the others. 
- We can evaluate each piece of data and immediately free up the memory space. 
- Data chunks would be small enough not to stress memory, but big enough for efficient use of the CPU.

In this notebook we will see how to divide the load between different processes.

In [7]:
import string

def words(file):
    """ Read a text file and return a sorted list 
    of (word, 1) values.
    """
    translator = str.maketrans('', '', string.punctuation)
    output = []
    with open(file) as f:
        for line in f:
            line = line.strip()
            line = line.translate(translator)
            for word in line.split():
                word = word.lower()
                output.append((word, 1))
    output.sort()
    return output

In [9]:
import operator
def reduce(words):
    """ Read the sorted list from map and print 
    out every word with its number of occurences"""
    d = {}
    for w in words:
        try:
            d[w[0]] +=1
        except KeyError:
            d[w[0]] = 1 
    
    return sorted(d.items(), key=operator.itemgetter(1), reverse=True)

In [10]:
reduce(words('sample.txt'))

[('magnam', 20),
 ('modi', 18),
 ('sed', 16),
 ('sit', 15),
 ('aliquam', 14),
 ('amet', 14),
 ('porro', 14),
 ('non', 13),
 ('quisquam', 13),
 ('ut', 13),
 ('numquam', 12),
 ('quaerat', 12),
 ('quiquia', 12),
 ('etincidunt', 11),
 ('ipsum', 11),
 ('eius', 10),
 ('tempora', 10),
 ('dolor', 9),
 ('velit', 9),
 ('voluptatem', 9),
 ('adipisci', 7),
 ('est', 7),
 ('labore', 7),
 ('dolorem', 6),
 ('neque', 6),
 ('consectetur', 5),
 ('dolore', 4)]

# Container datatypes

`collection` module implements specialized container datatypes providing alternatives to Python’s general purpose built-in containers, `dict`, `list`, `set`, and `tuple`.

- `namedtuple`	: factory function for creating tuple subclasses with named fields
- `deque`	: list-like container with fast appends and pops on either end
- `ChainMap`	: dict-like class for creating a single view of multiple mappings
- `Counter`	: dict subclass for counting hashable objects
- `defaultdict` :	dict subclass that calls a factory function to supply missing values


## Counter

A Counter is a dict subclass for counting hashable objects. It is an unordered collection where elements are stored as dictionary keys and their counts are stored as dictionary values. Counts are allowed to be any integer value including zero or negative counts.

Elements are counted from an iterable or initialized from another mapping (or counter):

In [2]:
from collections import Counter

violet = dict(r=23,g=13,b=23)
print(violet)
cnt = Counter(violet)  # or Counter(r=238, g=130, b=238)
print(cnt['c'])
print(cnt['r'])

{'r': 23, 'g': 13, 'b': 23}
0
23


In [3]:
print(*cnt.elements())

r r r r r r r r r r r r r r r r r r r r r r r g g g g g g g g g g g g g b b b b b b b b b b b b b b b b b b b b b b b


In [4]:
cnt.most_common(2)

[('r', 23), ('b', 23)]

In [5]:
cnt.values()

dict_values([23, 13, 23])

### Exercise 2.1

Use a `Counter` object to count words occurences in a `text` produced by the `lorem` module. Hint: use the the `most_common` method of `Counter` class.

The Counter class is similar to bags or multisets in some Python libraries or other languages. We will see later how to use Counter-like objects in a parallel context. 

## Partition data

In order to parallelize **reduce** operation, 
data must be aligned in a container. For this operation we will use the
`dict` subclass `defaultdict`.

![domain decomposition](https://computing.llnl.gov/tutorials/parallel_comp/images/domain_decomp.gif)

## defaultdict

This container is a `dict` subclass that calls a factory function to supply missing values.
Using list as the default_factory, it is easy to group a sequence of key-value pairs into a dictionary of lists:





In [11]:
from collections import defaultdict
s = [('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)]
d = defaultdict(list)
for k, v in s:
    d[k].append(v)

sorted(d.items())

[('blue', [2, 4]), ('red', [1]), ('yellow', [1, 3])]

### Exercise 2.2

- Use `int` as default_factory instead of list in the example above. The second part of every item of the class will be an integer instead of a list. You must replace the `append` by the suitable operator.
- Use the defaultdict for counting words in a text created by lorem module:


### Exercise 2.3

Create a function named `partition` that stores the key/value pairs from `words` (function created in notebook 01) into a `defaultdict` from `collections` module. Output will be:
```python
[('word1', [1, 1]), ('word2', [1]), ('word3', [1, 1, 1])]
```

### Exercise 2.4
- [itertools.chain(*mapped_values)](https://docs.python.org/3.6/library/itertools.html#itertools.chain) could be used for treating consecutive sequences as a single sequence. 
- [operator](https://docs.python.org/3.6/library/operator.html).itemgetter(1)
Return a callable object that fetches item from its operand using the operand’s __getitem__() method. It could be used to sort results.
```python
>>> import itertools, operator
>>> fruits = [('apple', 3), ('banana', 2), ('pear', 5), ('orange', 1)]
>>> vegetables = [('endive', 2), ('spinach', 1), ('celery', 5), ('carrot', 4)]
>>> getcount = operator.itemgetter(1)
>>> print(list(map(getcount, itertools.chain(fruits,vegetables) )))
[3, 2, 5, 1, 2, 1, 5, 4]
>>> print(sorted(itertools.chain(fruits,vegetables), key=getcount))
[('orange', 1), ('spinach', 1), ('banana', 2), ('endive', 2), ('apple', 3), ('carrot', 4), ('pear', 5), ('celery', 5)]
```

Write the program with the map, partition and reduce steps to compute
the list of words with their number of occurences of files sample[0-7].txt 
created in notebook 01. Example of output:
```python
[('aliquam', 17),('voluptatem', 15),('tempora', 14),('sit', 13),
 ('quisquam', 13), ('non', 13),('eius', 13),('quiquia', 12), ('magnam', 12)]
 ```